In [4]:
import pandas as pd
import numpy as np
import matplotlib as plt
import os
import logging
import arviz as az
print(az.__version__)
from arviz_stats import loo_subsample

0.23.0


In [5]:
# Set up logging: messages will include timestamp, log level, and message content
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

PATH_MODEL = '/volumes/hyijie_psy/CPP_stage2_HDDM/mid_van_2019/model_fitted'
PATH_MODEL_FIt = '../results/ddm'
os.makedirs(PATH_MODEL_FIt, exist_ok=True)

MODEL_CONFIG = {
    'm2_ams': {
        'main_param': 'v_ams',
        'interactions': [
            'v_C(condition)[T.2]:ams',
            'v_C(condition)[T.3]:ams',
            'v_C(condition)[T.4]:ams'
        ]
    }
}

In [6]:
path_model_infdata = os.path.join(PATH_MODEL, 'm1_non.nc')
m_infdata = az.from_netcdf(path_model_infdata)

In [20]:
# Extract and save posterior and Loo value

loo_results = []

for model_name, config in MODEL_CONFIG.items():

    main_param = config['main_param']
    interaction_names = config['interactions']
    
    path_model_infdata = os.path.join(PATH_MODEL, model_name + '.nc')
    assert os.path.exists(path_model_infdata), f" nc file not found: {model_name}"
    
    logger.info(f"Processing model: {model_name}")

    # Extract and save Loo for all models
    m_infdata = az.from_netcdf(path_model_infdata)
    #loo = az.loo(m_infdata, pointwise=True)
    #loo_dict = {
    #        "model": main_param,
    #        "loo": loo.elpd_loo,
    #        "p_loo": loo.p_loo
    #    }
    #loo_results.append(loo_dict)

    # Save posterior data for all models
    posterior_dict = {}

    # Main effect
    if main_param in m_infdata.posterior:
        posterior_dict[main_param] = m_infdata.posterior[main_param].stack(sample=("chain", "draw")).values
    else:
        logger.error(f"Main parameter '{main_param}' not in {model_name}")
        continue
    # Interaction effect
    for inter in interaction_names:
        if inter in m_infdata.posterior:
            posterior_dict[inter] = m_infdata.posterior[inter].stack(sample=("chain", "draw")).values
        else:
            logger.warning(f"Interaction '{inter}' not found in model '{model_name}'")

    # Save data
    data_posterior = pd.DataFrame(posterior_dict)
    path_save_posterior = os.path.join(PATH_MODEL_FIt, f'{model_name}_posterior.csv')
    data_posterior.to_csv(path_save_posterior, index=False)

loo_df = pd.DataFrame(loo_results)
loo_df.to_csv(os.path.join(PATH_MODEL_FIt, "loo_comparison.csv"), index=False)

AssertionError:  nc file not found: m2_ams

In [8]:

path_model_infdata = os.path.join(PATH_MODEL, 'm2_slps.nc')
assert os.path.exists(path_model_infdata), f" nc file not found: {model_name}"

logger.info(f"Processing model: {model_name}")
# Extract and save Loo for all models
m_infdata = az.from_netcdf(path_model_infdata)

2025-12-29 15:34:56,559 - INFO - Processing model: m2_slps_bin


In [7]:
az.summary(m_infdata)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
a,0.933,0.029,0.879,0.990,0.000,0.000,17454.0,12628.0,1.0
a_std,0.126,0.024,0.087,0.171,0.000,0.000,11011.0,11067.0,1.0
a_subj.01,0.846,0.013,0.822,0.871,0.000,0.000,12862.0,13277.0,1.0
a_subj.02,0.987,0.022,0.946,1.032,0.001,0.001,2131.0,1325.0,1.0
a_subj.03,0.882,0.017,0.849,0.915,0.000,0.000,6466.0,9428.0,1.0
...,...,...,...,...,...,...,...,...,...
v_C(condition)[T.1.0]:slps_subj.16,-0.253,0.144,-0.539,0.002,0.002,0.001,6879.0,10064.0,1.0
v_C(condition)[T.1.0]:slps_subj.17,0.124,0.156,-0.159,0.420,0.002,0.001,7979.0,10836.0,1.0
v_C(condition)[T.1.0]:slps_subj.18,-0.072,0.137,-0.333,0.183,0.002,0.001,7420.0,11444.0,1.0
v_C(condition)[T.1.0]:slps_subj.19,0.261,0.145,-0.023,0.521,0.002,0.001,7838.0,11196.0,1.0


In [24]:
m_infdata

Inference data with groups:
	> posterior
	> posterior_predictive
	> log_likelihood
	> prior
	> observed_data

In [ ]:
loo_results = loo_subsample(m_infdata, observations=4, var_name="obs", pointwise=True)

AttributeError: module 'arviz.stats' has no attribute 'loo_subsample'

In [1]:
from arviz_stats import loo_subsample

ImportError: cannot import name 'loo_subsample' from 'arviz_stats' (/opt/miniconda3/envs/env4eeg/lib/python3.13/site-packages/arviz_stats/__init__.py)